In [1]:
FILE_PATH = '/home/matt/repos/ohia.ai/data'

model_name = 'mobilenetv2'

training_type = 0
# 0: finetune using pretrained on ImageNet
# 1: pretrain on PlantNet
# 2: finetune using pretrained on PlantNet

seed = 1
batch_size = 32

filtered = False
augmentation = False

n_thread = 32
gpu = 0

save_model = False    

In [2]:
import os, re, glob, json
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']=str(gpu)

import numpy as np
import pandas as pd

from keras import callbacks
from sklearn.model_selection import train_test_split

from ohia.encoders import FastLabelEncoder
from ohia.utils import PlantNetGenerator, make_dir

Using TensorFlow backend.


In [3]:
def get_model_path(model_name, training_type, seed, batch_size, augmentation):
    """ Get path for given model parameters
    
    Args:
        model_name: Name of neural network architecture, one of
          "mobilenetv1", "mobilenetv2", "nasnetmobile"
        training_type: Integer encoding:
            * if 0 fine-tune
            * if 1 pretrain
            * if 2 load pretrained then fine-tune.
        seed: Random seed.
        batch_size: Number of observations needed before updating weights.
        augmentation: Boolean flag.  If true then perform data augmentation.
    """
    model_path = f'{FILE_PATH}/models/'
    
    # fine-tuning
    if training_type == 0:
        model_path += f'finetuned_{model_name}'
        model_path += '_with_augmention' if augmentation else ''
        
    # pretraining
    elif training_type == 1:
        model_path += f'pretrained_{model_name}'
        model_path += '_with_augmention' if augmentation else ''
        
    # pretraining then fine-tuning
    elif training_type == 2:
        model_path += f'finetuned_{model_name}'
        model_path += '_with_augmention' if augmentation else ''
        model_path += '_using_plantnet_pretrained'
    else :
        raise ValueError('training_type must be either 0, 1, or 2.') 
            
    model_path += f'_seed-{seed}_batch_size-{batch_size}'
    return model_path

In [4]:
def get_model(model_name, n_classes, training_type, pretrain_file=None):
    """ Get ohiaNet architecture
    
    Args:
        model_name: Name of neural network architecture, one of
          "mobilenetv1", "mobilenetv2", "nasnetmobile"
        n_classes: Numeber of output units
        training_type: Integer encoding:
            * if 0 fine-tune
            * if 1 pretrain
            * if 2 load pretrained then fine-tune.
            
    Returns: Keras model.
    """        
    from keras import layers, Model
    from keras.optimizers import Adam
    from keras.applications.mobilenet import MobileNet
    from keras.applications.mobilenetv2 import MobileNetV2
    from keras.applications.nasnet import NASNetMobile
    from ohia.metrics import top_1_accuracy, top_3_accuracy, top_5_accuracy

    import keras.backend as K
    K.clear_session()

    # load pretrained ImageNet model
    if model_name == 'mobilenetv1':
        base_model = MobileNet(
            input_shape=(224,224,3),
            weights='imagenet',
            include_top=False
        )        
    elif model_name == 'mobilenetv2':
        base_model = MobileNetV2(
            input_shape=(224,224,3),
            alpha=1.4,
            weights='imagenet',
            include_top=False
        )
    elif model_name == 'nasnetmobile':
        base_model = NASNetMobile(
            input_shape=(224,224,3),
            weights='imagenet',
            include_top=False
        )
    else:
        assert ValueError(
            f'model_name parameter must be one of the following'
            ' "mobilenetv1",'
            ' "mobilenetv2",'
            ' "nasnetmobile"'
        )

        
    if training_type==0:
        
        # map ImageNet features to plants
        x = base_model.output
        x = layers.GlobalAveragePooling2D()(x)
        outputs = layers.Dense(n_classes, activation='softmax')(x)
        model = Model(inputs=base_model.input, outputs=outputs)        
        
        # freeze all but the last layer
        for layer in model.layers[:-1]:
            layer.trainable = False

    elif training_type==1:
        
        # map ImageNet features to plants
        x = base_model.output
        x = layers.GlobalAveragePooling2D()(x)
        outputs = layers.Dense(n_classes, activation='softmax')(x)
        model = Model(inputs=base_model.input, outputs=outputs)        
                    
    elif training_type==2:
        
        N_PLANTNET_CLASSES = 436
            
        # map ImageNet features to plants
        x = base_model.output
        x = layers.GlobalAveragePooling2D()(x)
        outputs = layers.Dense(N_PLANTNET_CLASSES, activation='relu')(x)

        # load the model
        pretrained_model = Model(inputs=base_model.input, outputs=outputs)
        pretrained_model.load_weights(pretrain_file)
        
        # add final layer
        outputs = layers.Dense(n_classes, activation='softmax')(pretrained_model.output)
        model = Model(inputs=pretrained_model.input, outputs=outputs)

        # freeze all but the last two layers
        if training_dataset == 'scraped':
            for layer in model.layers[:-2]:
                layer.trainable = False
            
    else :
        raise ValueError('training_type must be either 0, 1, or 2.') 
                
    # compile the model
    model.compile(
        optimizer=Adam(lr=0.001),
        loss='categorical_crossentropy',
        metrics=[top_1_accuracy, top_3_accuracy, top_5_accuracy]
    )

    return model

In [5]:
# make directory for current expirement
model_path = get_model_path(model_name, 1, seed, batch_size, augmentation)
make_dir(model_path)

# get pretrained file
pretrain_path = get_model_path(model_name, 1, seed, batch_size, augmentation)
pretrain_file = np.sort(glob.glob(f'{pretrain_path}/**.h5'))[-1]

# get list of images and labels
image_dir = 'plantnet_filtered'if training_type==2 else 'scraped_filtered'
file_list = glob.glob(f'{FILE_PATH}/preprocessed_images/{image_dir}/**/*.jpg', recursive=True)
full_label_list = [re.split('/', f)[-2] for f in file_list]

# encode label names with ids
fle = FastLabelEncoder()
label_ids = fle.fit_transform(full_label_list)

# save id2label map
id2label = {int(fle.transform([label])):label for label in np.unique(full_label_list)}
with open(f'{model_path}/plantnet_classes.json', 'w') as fp:
    json.dump(id2label, fp)

# split data
train_files, valid_files, train_ids, valid_ids \
    = train_test_split(file_list, label_ids, test_size=0.1, random_state=seed)

In [6]:
# create generators
n_classes = len(np.unique(full_label_list))
train_generator = PlantNetGenerator(
    train_files, train_ids, n_classes,
    batch_size=batch_size,
    augment=augmentation
)
valid_generator = PlantNetGenerator(
    valid_files, valid_ids, n_classes,
    batch_size=batch_size,
    augment=augmentation,
    shuffle=False
)

In [7]:
# define callbacks
callbacks_list = [
    callbacks.EarlyStopping(
        monitor='val_top_3_accuracy',
        patience=10,
        verbose=1,
        mode='max',
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_top_3_accuracy',
        factor=0.25,
        patience=2,
        verbose=1,
        mode='max',
    ),
    callbacks.ModelCheckpoint(
        monitor='val_top_3_accuracy',
        filepath=f'{model_path}/weights' + '_{epoch:02d}.h5',
        save_best_only=True,
        save_weights_only=False,
        mode='max',
    ),
]

In [8]:
# train model 
model = get_model(model_name, n_classes, training_type, pretrain_file)
model.fit_generator(
    generator=train_generator,
    validation_data=valid_generator,
    callbacks=callbacks_list,
    use_multiprocessing=True,
    workers=n_thread,
    epochs=100,    
)

Epoch 1/100
485/485 [==============================] - 67s 138ms/step - loss: 1.8678 - top_1_accuracy: 0.4934 - top_3_accuracy: 0.6988 - top_5_accuracy: 0.7840 - val_loss: 2.0024 - val_top_1_accuracy: 0.4699 - val_top_3_accuracy: 0.6940 - val_top_5_accuracy: 0.7777
Epoch 2/100
485/485 [==============================] - 58s 120ms/step - loss: 1.2614 - top_1_accuracy: 0.6377 - top_3_accuracy: 0.8314 - top_5_accuracy: 0.8962 - val_loss: 1.9410 - val_top_1_accuracy: 0.4982 - val_top_3_accuracy: 0.7170 - val_top_5_accuracy: 0.7960
Epoch 3/100
485/485 [==============================] - 57s 118ms/step - loss: 1.0870 - top_1_accuracy: 0.6784 - top_3_accuracy: 0.8671 - top_5_accuracy: 0.9227 - val_loss: 1.9030 - val_top_1_accuracy: 0.5130 - val_top_3_accuracy: 0.7258 - val_top_5_accuracy: 0.8202
Epoch 4/100
485/485 [==============================] - 61s 126ms/step - loss: 0.9801 - top_1_accuracy: 0.7097 - top_3_accuracy: 0.8858 - top_5_accuracy: 0.9369 - val_loss: 2.0250 - val_top_1_accuracy: 0

In [9]:
# save results
results = pd.DataFrame(model.history.history)
results.to_csv(f'{model_path}/results.csv', index=False)

In [10]:
# print best results
print(results.iloc[results.val_top_3_accuracy.values.argmax()])

val_loss              1.972768
val_top_1_accuracy    0.522995
val_top_3_accuracy    0.737028
val_top_5_accuracy    0.825472
loss                  0.836548
top_1_accuracy        0.741559
top_3_accuracy        0.911469
top_5_accuracy        0.954510
lr                    0.001000
Name: 5, dtype: float64


In [12]:
# save best model
if save_model:
    import tensorflowjs as tfjs
    model = get_model(model_name, n_classes)
    best_weights = glob.glob(f'{model_path}/**.h5')
    best_weights = np.sort(best_weights)[-1]
    model.load_weights(best_weights)
    tfjs.converters.save_keras_model(model, f'{model_path}/tfjs')